## Сентимент-аналіз із використанням "мішка слів"

Для сентимент-аналізу я витягнув користувацькі відгуки з кількох категорій розділу "Побутова техніка" на Розетці. Код для витягування міститься у файлі prepare_data.py. Там же здійснено первинну обробку даних (наприклад, видалено слова "Недоліки" та "Переваги", які по шаблону від "Розетки" містяться майже в усіх відгуках) та відокремлено україномовні відгуки, використовуючи бібліотеку langdetect.

Більша частина коду виведена в окремі файли в цій директорії, а тут я лише продемонструю, як код працює і які дає результати. Для початку створимо тренувальний та тестувальний датасети.

In [1]:
from build_datasets import df, build_datasets, stopwords

In [2]:
# set the seed for all subsequent random states
seed = 505

In [3]:
train, test = build_datasets(df, random_state=seed)

Для початку трохи про дані. Я загалом практично не дивився на "нейтральні" відгуки: більшості людей на розетці продукт або подобається (навіть якщо з деякими недоліками), або не подобається, при чому 3 бали - це вже негатив, попри "серединність" цієї оцінки (хто з нас купив би продукт із середнім балом 3 із 5?). Крім того, і це мене здивувало, негативних відгуків (оцінка 3 і нижче) - тільки 11.5% від загальної кількості відгуків:

In [4]:
df['sent'].value_counts()

pos    7339
neg     952
Name: sent, dtype: int64

Незбалансованість класів означає проблеми при класифікації, як ми побачимо далі.

### Naive Bayes v.1

Найпростіший класифікатор на основі мішка слів з використанням Наївного Баєса, без використання стоп-слів, лем і т.д.:

In [5]:
from baseline_classifier import prob_dict, priors, baseline_classifier
from sklearn.metrics import classification_report

In [6]:
test['base_pred'] = test['text'].apply(
    lambda text: baseline_classifier(text, prob_dict, priors))

print(classification_report(test['sent'], test['base_pred']))

             precision    recall  f1-score   support

        neg       0.00      0.00      0.00       281
        pos       0.89      1.00      0.94      2206

avg / total       0.79      0.89      0.83      2487



Жоден відгук не було класифіковано як негативний, що робить цей класифікатор totally useless.

### Тональний класифікатор

Наступний крок - спробувати тональний словник. Цей класифікатор сумує тональність усіх слів відгуку (беручи леми слів), які мають хоч якусь тональність, і класифікує як негативні ті відгуки, які мають мінусову сумарну тональність. Стоп-слова не враховуються.

In [7]:
from tonal_classifier import tonal_dict, tonal_classifier

test['tonal_pred'] = test['text'].apply(
    lambda text: tonal_classifier(text, tonal_dict, stopwords))

print(classification_report(test['sent'], test['tonal_pred']))

             precision    recall  f1-score   support

        neg       0.27      0.35      0.31       281
        pos       0.91      0.88      0.90      2206

avg / total       0.84      0.82      0.83      2487



Цього разу ми знайшли понад третину негативних відгуків, але recall та F1 для негативних відгуків усе ще дуже низькі.

### Naive Bayes v.2


Наступний крок - Наївний Баєс версія 2, імплементований з такими особливостями:

- враховано стоп-слова (перелік знайдено в інтернеті);
- беруться леми слів (як їх подає бібліотека pymorphy2);
- кожне слово у відгуці рахується лише раз (якщо "поганий" згадано 5 разів в одному відгуці, ми рахуємо його тільки раз) - за рекомендацією Джурафскі і Мартіна;
- усі слова після "не" і до найближчого розділового знака ми перетворюємо на спеціальні токети з "НЕ" на початку: "не дуже хороший" стає "не", "НЕ\_дуже", "НЕ\_хороший" - це теж рекомендація з підручника.

In [8]:
from naive_bayes import build_vocab, get_priors, train_Naive_Bayes, NB_classifier

vocab = build_vocab(train, 'sent', 'text', stopwords)
priors = get_priors(train, 'sent')
prob_dict = train_Naive_Bayes(vocab, ['pos', 'neg'])

test['NB_pred'] = test['text'].apply(
    lambda text: NB_classifier(text, prob_dict, priors, stopwords))

print(classification_report(test['sent'], test['NB_pred']))

             precision    recall  f1-score   support

        neg       0.00      0.00      0.00       281
        pos       0.89      1.00      0.94      2206

avg / total       0.79      0.89      0.83      2487



/home/igor/Bin/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


І у нас знову жоден відгук не класифікується як негативний! Усе-таки головна проблема, вочевидь, у незбалансованих класах у тренувальній вибірці. 

Якщо ми зробимо даунсемплінг для позитивних відгуків таким чином, щоб їх у тренувальному сеті лишилась тільки половина, то класифікатор зможе краще "бачити" негативні відгуки у тестувальній вибірці (яка, звісно, повинна лишатись незбалансованою, як це є в реальному житті). Мінус - ми втрачаємо дані, яких і так не надто багато. Але ми можемо використати ці дані у тестовій вибірці - вона буде більшою, ніж тренувальна, але це зовсім не проблема, тому що тест-сет має показувати результати в "реальності", яка завжди більша за будь-який трейн-сет :) 

Опція balanced у функції build_datasets будує збалансований тренувальний датасет (порівну обох класів), а потім із усіх інших даних робить тестову вибірку таким чином, щоб відсоток обох класів збігався з відсотком у початкових даних.

In [9]:
train, test = build_datasets(df, random_state=seed, balanced=True)
print(len(train))
print(len(test))

952
4232


In [10]:
vocab = build_vocab(train, 'sent', 'text', stopwords)
priors = get_priors(train, 'sent')
prob_dict = train_Naive_Bayes(vocab, ['pos', 'neg'])

test['NB_pred'] = test['text'].apply(
    lambda text: NB_classifier(text, prob_dict, priors, stopwords))

print(classification_report(test['sent'], test['NB_pred']))

             precision    recall  f1-score   support

        neg       0.45      0.76      0.56       486
        pos       0.97      0.88      0.92      3746

avg / total       0.91      0.86      0.88      4232



У нас усе ще низький precision, але непогане покриття і F1 для негативних відгуків, що дозволяє отримати також найвищий поки що загальний F1.

### Perceptron

Другий алгоритм класифікації, який я імплементував "своїми силами" - перцептрон.

In [11]:
from perceptron import initialize_vocab, train_perceptron, perceptron_classifier

perc_vocab = initialize_vocab(vocab) # using vocabulary created before
trained_vocab = train_perceptron(100, train, perc_vocab, stopwords)
# 100 iterations takes quite some time to complete

In [12]:
test['perc_pred'] = test['text'].apply(
    lambda text: perceptron_classifier(text, trained_vocab, stopwords))

print(classification_report(test['sent'], test['perc_pred']))

             precision    recall  f1-score   support

        neg       0.30      0.78      0.44       486
        pos       0.96      0.77      0.86      3746

avg / total       0.89      0.77      0.81      4232



Плюс перцептрона - поки що найкраще покриття для негативних відгуків. У деяких задачах цей плюс може перевершувати мінуси. А мінуси - повільність та гірші результати в усіх інших показниках.

### Класифікація в sklearn

Тепер можна спробувати класифікувати ті самі дані за допомогою класифікаторів "з коробки", яких вдосталь у бібліотеці scikit-learn. Для кожного з класифікаторів ми застосуємо підхід "мішок слів", із токенізацією від бібліотеки tokenize_uk та тим самим списком стоп-слів, що і раніше. Також використаємо TF-IDF-трансформацію, яку вміє автоматично робити sklearn (вона додає дуже мало покращення до результатів, і я не застосовував цю трансформацію до "своїх" класифікаторів).

In [13]:
from tokenize_uk.tokenize_uk import tokenize_words

from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.naive_bayes import MultinomialNB, BernoulliNB
from sklearn.linear_model import SGDClassifier, LogisticRegression

Спершу - две версії Наївного Баєса, одна з урахуванням повторюваності слів у документі, інша - без (подібно до того, як я рахував кожне слово в документі тільки один раз).

In [14]:
sklearn_NB = Pipeline([('vect', CountVectorizer(tokenizer=tokenize_words,
                                             stop_words=stopwords)),
                      ('tfidf', TfidfTransformer()),
                      ('clf', MultinomialNB())])
sklearn_NB.fit(train['text'], train['sent'])

test['sklearn_NB_pred'] = sklearn_NB.predict(test['text'])

print(classification_report(test['sent'], test['sklearn_NB_pred']))

             precision    recall  f1-score   support

        neg       0.44      0.70      0.54       486
        pos       0.96      0.89      0.92      3746

avg / total       0.90      0.87      0.88      4232



In [15]:
bernoulli = Pipeline([('vect', CountVectorizer(tokenizer=tokenize_words,
                                             stop_words=stopwords)),
                      ('tfidf', TfidfTransformer()),
                      ('clf', BernoulliNB())])
bernoulli.fit(train['text'], train['sent'])

test['bernoulli_pred'] = bernoulli.predict(test['text'])
print(classification_report(test['sent'], test['bernoulli_pred']))

             precision    recall  f1-score   support

        neg       0.49      0.53      0.51       486
        pos       0.94      0.93      0.93      3746

avg / total       0.89      0.88      0.88      4232



Середній F1 близький до мого Наївного Баєса (залежно від параметру seed на початку, інколи мій класифікатор показує трохи кращі результати, інколи навпаки), що логічно.

Як щодо логістичної регресії?

In [16]:
logistic = Pipeline([('vect', CountVectorizer(tokenizer=tokenize_words,
                                             stop_words=stopwords)),
                      ('tfidf', TfidfTransformer()),
                      ('clf', LogisticRegression(random_state=seed))])

logistic.fit(train['text'], train['sent'])

test['logistic_pred'] = logistic.predict(test['text'])
print(classification_report(test['sent'], test['logistic_pred']))

             precision    recall  f1-score   support

        neg       0.35      0.78      0.48       486
        pos       0.97      0.81      0.88      3746

avg / total       0.89      0.81      0.83      4232



Результати погіршились, хоча покриття для негативних відгуків хороше.

SVM?

In [17]:
svm = Pipeline([('vect', CountVectorizer(tokenizer=tokenize_words,
                                             stop_words=stopwords)),
                      ('tfidf', TfidfTransformer()),
                      ('clf', SGDClassifier(tol=1e-3, max_iter=100,
                                           random_state=seed))])

svm.fit(train['text'], train['sent'])

test['svm_pred'] = svm.predict(test['text'])
print(classification_report(test['sent'], test['svm_pred']))

             precision    recall  f1-score   support

        neg       0.30      0.85      0.44       486
        pos       0.97      0.75      0.84      3746

avg / total       0.90      0.76      0.80      4232



Результати схожі. Я пробував деякі (далеко не всі можливі) комбінації параметрів, і не схоже що щось суттєво змінилось би. 

Загалом виглядає так, що дані, які в мене є, і їхня порівняно невелика кількість роблять Наївний Баєс із застосуванням кількох методів обробки фіч (лематизація, стоп-слова, врахування негацій), і зі збалансованим тренувальним датасетом, найкращим методом класифікації відгуків на позитивні та негативні в межах парадигми "мішка слів".

### Бонус

У нашому датасеті є окремо витягнуті тексти переваг і тексти недоліків товарів. Логічно, що в позитивних відгуках буде більше тексту про переваги, а в негативних - навпаки. Це можна перевірити на наших даних, попередньо замінивши відсутні переваги або недоліки на прочерк.

In [18]:
df2 = df.copy()
df2['adv'] = df2['adv'].fillna('-')
df2['disadv'] = df2['disadv'].fillna('-')
df2['len_adv'] = df2['adv'].apply(len)
df2['len_disadv'] = df2['disadv'].apply(len)
df2.groupby('sent')[['len_adv', 'len_disadv']].mean()

,len_adv,len_disadv
sent,,
neg,21.514706,52.358193
pos,35.413135,23.371440


У негативних відгуках різниця в довжині тексту дуже велика, у позитивних - помірна. Цей факт можна використати для примітивного класифікатора, який позначає позитивними всі відгуки, де переваги довші за недоліки, і навпаки для негативних. Я не ділив дані на тренувальну і тестувальну вибірку, тому що тут нема ніякого тренування.

In [19]:
def classify_by_len(row):
    len_adv = row['len_adv']
    len_disadv = row['len_disadv']
    if len_adv >= len_disadv:
        return 'pos'
    else:
        return 'neg'
    
df2['len_pred'] = df2.apply(classify_by_len, axis=1)
print(classification_report(df2['sent'], df2['len_pred']))

             precision    recall  f1-score   support

        neg       0.28      0.66      0.39       952
        pos       0.95      0.78      0.85      7339

avg / total       0.87      0.76      0.80      8291



Результати далеко не найгірші :)

### Що я не робив

- нейтральні відгуки. Було б складніше побудувати збалансований тренувальний + "великий" тестувальний датасет, і в будь-якому разі незрозуміло, чи справді відгуки, які мають 3 або 4 бали, можна назвати нейтральними;
- текст лише до початку шаблону "Переваги: ... Недоліки: ...". Нерідко в недоліках або перевагах просто слова "нема" або "ціна", що без контексту, по ідеї, тільки додає шум. Але спроби класифікувати без цього тексту дають трохи гірші результати, тому що все-таки частіше в шаблон вписують змістовний текст, або не пишуть ніякий текст перед шаблоном.